In [81]:
Sys.setlocale(category = "LC_ALL", locale = "en_US.UTF-8")

[1] "en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/C"

# Dependencies

In [82]:
pacman::p_load(dplyr, ggplot2, ggbeeswarm, ggpubr, ggsignif, cowplot, ComplexHeatmap, factoextra, FactoMineR, VIM, missMDA) 

# Functions

In [83]:
source("/Users/anabbi/git/ped_CapTCRseq/R/ggplot2_theme.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/color_schemes.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/Misc_functions.R")

# Paths

In [84]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"
gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [85]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/"

# Main

In [6]:
pca.fx <- function(df, cols, np) {
    mymat <- df[, cols]
    rownames(mymat) <- df[["sample_id"]]
    # remove rows with all NAs
    mymat <- mymat[rowSums(is.na(mymat)) != ncol(mymat), ]
    # rmeove those with only 0s and NAs
    mymat <- mymat[rowSums(mymat, na.rm = TRUE) != 0, ]
    # order df by sample_id
    df <- df[rownames(mymat), ]

    # remove all NA values
    mymat_narm <- mymat[rowSums(is.na(mymat)) == 0, ]
    df_narm <- df[rownames(mymat_narm), ]
    # get vars and include only those with variance > 0
    myvars <- sort(Rfast::colVars(mymat_narm), decreasing = TRUE)
    mymat_narm <- mymat_narm[, names(myvars)[myvars > 0]]
    # PCA
    pca <- prcomp(mymat_narm, center = TRUE, scale. = TRUE)
    # plot PCA with factoextra
    p1 <- fviz_pca_ind(pca,
        col.ind = as.vector(df_narm$cancergroup),
        # gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
        # remove labels
        label = "none",
        repel = TRUE, addEllipses = FALSE
    )
    # scree plot
    p2 <- fviz_eig(pca)
    myx <- pca$x
    mydist <- as.matrix(dist(myx[, 1:np]))
    # print(head(mydist) )
    sample_df_narm <- df_narm %>%
        group_by(Patient) %>%
        # order cyclenum
        arrange(cyclenum) %>%
        ungroup()
    pts <- names(which(table(sample_df_narm$Patient) > 1))
    # print(head(sample_df_narm[,1:10]) )
    sample_df_narm$dist <- NA
    for (i in pts) {
        tmp <- sample_df_narm[sample_df_narm$Patient == i, ]
        distances_df <- mydist[tmp$sample_id, tmp$sample_id]
        if (nrow(tmp) == 2) {
            mydiff <- diff(tmp$cyclenum)
            if (mydiff == 1) {
                tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
            }
        }
        if (nrow(tmp) == 3) {
            mydiff <- diff(tmp$cyclenum)
            if (mydiff[1] == 1) {
                tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
            }
            if (mydiff[2] == 1) {
                tmp$dist[3] <- distances_df[tmp$sample_id[2], tmp$sample_id[3]]
            }
        }
        if (nrow(tmp) == 4) {
            mydiff <- diff(tmp$cyclenum)
            if (mydiff[1] == 1) {
                tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
            }
            if (mydiff[2] == 1) {
                tmp$dist[3] <- distances_df[tmp$sample_id[2], tmp$sample_id[3]]
            }
            if (mydiff[3] == 1) {
                tmp$dist[4] <- distances_df[tmp$sample_id[3], tmp$sample_id[4]]
            }
        }
        if (nrow(tmp) == 5) {
            mydiff <- diff(tmp$cyclenum)
            if (mydiff[1] == 1) {
                tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
            }
            if (mydiff[2] == 1) {
                tmp$dist[3] <- distances_df[tmp$sample_id[2], tmp$sample_id[3]]
            }
            if (mydiff[3] == 1) {
                tmp$dist[4] <- distances_df[tmp$sample_id[3], tmp$sample_id[4]]
            }
            if (mydiff[4] == 1) {
                tmp$dist[5] <- distances_df[tmp$sample_id[4], tmp$sample_id[5]]
            }
        }
        sample_df_narm$dist[sample_df_narm$Patient == i] <- tmp$dist
    }
    # message("sample_df_narm")
    # print(head(sample_df_narm))
    allpts <- as.data.frame(table(sample_df_narm$Patient), stringsAsFactors = F)
    sample_df_narm$dist[sample_df_narm$dist == 0] <- NA
    allpts$sumdist <- NA
    allpts$meddist <- NA
    for (i in 1:nrow(allpts)) {
        tmp <- as.data.frame(sample_df_narm[sample_df_narm$Patient == allpts$Var1[i], ])
        allpts$sumdist[i] <- sum(tmp[tmp$dist != 0, "dist"], na.rm = T)
        allpts$meddist[i] <- median(tmp[tmp$dist != 0, "dist"], na.rm = T)
    }
    allpts$sumdist_n <- allpts$sumdist / allpts$Freq
    allpts$meddist_n <- allpts$meddist / allpts$Freq
    # message("allpts")
    # print(head(allpts))
    allpts_no0 <- allpts[allpts$sumdist_n > 0, ]
    allpts_no0 <- allpts_no0[allpts_no0$Freq > 1, ]
    # message("allpts_no0")
    # print(dim(allpts_no0))

    mysample_df <- sample_df_narm[sample_df_narm$Patient %in% allpts_no0$Var1, ]
    # message("mysample_df")
    # print(head(mysample_df))
    # dim(mysample_df)
    mypatients <- mysample_df[!duplicated(mysample_df$Patient), ]
    # message("mypatients")
    # print(head(mypatients))
    # dim(mypatients)
    # head(mypatients)
    # head(allpts_no0)
    mypatients$sumdist_n <- allpts_no0$sumdist_n[match(mypatients$Patient, allpts_no0$Var1)]
    mypatients$sumdist <- allpts_no0$sumdist[match(mypatients$Patient, allpts_no0$Var1)]
    mypatients$meddist_n <- allpts_no0$meddist_n[match(mypatients$Patient, allpts_no0$Var1)]
    mypatients$meddist <- allpts_no0$meddist[match(mypatients$Patient, allpts_no0$Var1)]
    # print(head(mypatients) )
    myfit_sum <- rms::lrm(Relapse ~ sumdist_n, data = mypatients)
    myfit_med <- rms::lrm(Relapse ~ meddist_n, data = mypatients)
    return(list(pca, p1, p2, mypatients, myfit_sum, myfit_med))
}


In [7]:
pbmc_hla_v_j_gliph <- readr::read_rds(paste0(datapath, "/int_pbmc_hla_v_j_gliph.rds"))

In [8]:
rownames(pbmc_hla_v_j_gliph) <- pbmc_hla_v_j_gliph$sample_id

In [9]:
dim(pbmc_hla_v_j_gliph)

[1]  534 8829

In [10]:
table(pbmc_hla_v_j_gliph$Relapse)


  0   1 
431 103 

In [11]:
cols_tbv <- colnames(pbmc_hla_v_j_gliph)[which(grepl("TRBV", colnames(pbmc_hla_v_j_gliph)) ) ]
cols_tbj <- colnames(pbmc_hla_v_j_gliph)[which(grepl("TRBJ", colnames(pbmc_hla_v_j_gliph)) ) ]
cols_flow <- c('ATC', 'Naïve%','SCM%','CM%','EM%','TE%','PD1%','LAG3%','TIM3%')
cols_others <- c("log10shann")
cols_hla <- colnames(pbmc_hla_v_j_gliph)[which(grepl(":0", colnames(pbmc_hla_v_j_gliph)) ) ]

In [12]:
which(grepl("TRBJ", colnames(pbmc_hla_v_j_gliph) ) )

[1] 261 262 263 264 265 266 267 268 269 270 271 272 273

In [13]:
colnames(pbmc_hla_v_j_gliph)[274:ncol(pbmc_hla_v_j_gliph)]

[1] "SLG%E"        "S%GGNTE"      "GIWV"         "IWV"         
   [5] "WVYE"         "IWVY"         "YDTG"         "S%TGNTE"     
   [9] "S%SGANV"      "SV%GNTE"      "EEEG"         "S%GQGNTE"    
  [13] "SLQG%E"       "S%VNTE"       "SVG%NTE"      "SV%GGTE"     
  [17] "S%GLAGGE"     "S%GNYG"       "S%GSGE"       "%QGNTE"      
  [21] "S%GTYE"       "S%GLE"        "SP%GTE"       "S%TLNTE"     
  [25] "S%RTGE"       "RG%GNTE"      "TGIW"         "S%QGTGE"     
  [29] "R%STDT"       "S%GQMNTE"     "S%GSYG"       "SVGG%TE"     
  [33] "SP%MNTE"      "%GTGNTE"      "SV%GNE"       "SIDK"        
  [37] "MTNT"         "SV%GDE"       "SRT%NTE"      "S%GNTE"      
  [41] "SIGG%E"       "SV%VNTE"      "SP%GNT"       "%GGSYE"      
  [45] "S%ENTE"       "SPGQG%TE"     "SPG%TGE"      "S%TVNTE"     
  [49] "SA%NTE"       "SV%GTGE"      "SG%SNQP"      "SLG%NTE"     
  [53] "S%GGSYNE"     "SPG%NE"       "%TGMNTE"      "SLGQG%YE"    
  [57] "S%GMNTE"      "S%GQGTYE"     "S%QGRTE"      "SGG%TDT"     
  [61] "S%DRSYE"      "ST%GNTE"      "SLGTG%NE"     "S%GDSNQP"    
  [65] "SV%GSGANV"    "S%RSNQP"      "SL%PNTGE"     "S%GSYE"      
  [69] "SPGQ%NTE"     "R%QGNTE"      "SE%NTE"       "S%GTSTDT"    
  [73] "T%GNTE"       "S%GGTNTE"     "S%TSGAYNE"    "SR%ENTE"     
  [77] "S%LGTE"       "SLS%E"        "SL%SSGNT"     "STGD%NQP"    
  [81] "SVLG%E"       "RG%GGTE"      "SQDR%YE"      "SL%TNTGE"    
  [85] "%TGGNQP"      "RGQG%TE"      "S%DRGSYE"     "%DRGNTE"     
  [89] "S%RGGTE"      "S%TMNTE"      "S%DRAYE"      "SS%SGNT"     
  [93] "S%DRLNTE"     "SL%DE"        "SPT%NTE"      "SPGR%NTE"    
  [97] "S%TENTE"      "S%GGSTE"      "SPSR%E"       "S%RSTGE"     
 [101] "SRTSG%YNE"    "S%GTNTE"      "S%GRTDT"      "S%GTSGRNE"   
 [105] "SL%KNTE"      "S%QGRGTE"     "SLS%SGNT"     "SRG%GYE"     
 [109] "RDR%NTE"      "RG%MNTE"      "SP%GTDT"      "S%RGYG"      
 [113] "SDR%YG"       "S%RSSYE"      "SDR%NTE"      "%RGMNTE"     
 [117] "SYR%GE"       "%RSSYNE"      "SDR%TGE"      "S%SGSE"      
 [121] "S%GTGDE"      "S%DSGNT"      "SPG%NEK"      "S%NSGNT"     
 [125] "SPG%MNTE"     "SP%RGNTE"     "SRGQG%YE"     "SPRQG%NTE"   
 [129] "TG%SYE"       "SD%GYE"       "SVVG%E"       "%GMNTE"      
 [133] "SP%RNTE"      "ST%GTE"       "S%AGLNTE"     "STGT%YNE"    
 [137] "S%GVNE"       "R%GGNTE"      "SYSRG%E"      "S%GTGSGNT"   
 [141] "SY%GTE"       "%VSGANV"      "%DGNYG"       "SP%GYG"      
 [145] "S%RGGYNE"     "SPG%GTE"      "S%PNTGE"      "SL%GE"       
 [149] "SLG%GMNTE"    "S%SSYNE"      "SYG%YE"       "SPGG%ET"     
 [153] "SL%GQGNE"     "SSGTG%TGE"    "SS%MNTE"      "SPG%GET"     
 [157] "S%TSNE"       "S%QGATE"      "SR%GGNTE"     "SLDG%NTE"    
 [161] "SLGTG%NTE"    "%QTNTE"       "SVQ%NQP"      "SE%GMNTE"    
 [165] "S%GTGQET"     "T%SSYNE"      "SV%PNQP"      "SQD%AYE"     
 [169] "G%GGTNEK"     "SS%GMNTE"     "SAGQ%NTE"     "S%TSGPYNE"   
 [173] "R%GGNT"       "SSG%LNTE"     "SYDD"         "S%SGE"       
 [177] "SF%GNTE"      "S%QGYSNQP"    "SPDR%NEK"     "SPGQ%SGNT"   
 [181] "SP%GGET"      "SR%RGNTE"     "SYS%YE"       "%PGGTE"      
 [185] "S%PNTE"       "S%ARGET"      "SPLG%NTE"     "%RTGNTE"     
 [189] "S%SYE"        "S%GLTYE"      "SYS%ET"       "S%AGANTGE"   
 [193] "DR%NTGE"      "%DSLNTE"      "G%GSYNE"      "SQDR%T"      
 [197] "S%RGNTE"      "%DRPNTE"      "SST%NTE"      "S%MNTE"      
 [201] "S%GGDG"       "R%GNTE"       "%TGGSYE"      "S%RDSNQP"    
 [205] "S%AGYE"       "SRDG%NTE"     "S%GGGYE"      "%PGQGNTE"    
 [209] "SL%RLNTE"     "SP%GPNTE"     "SE%ASTDT"     "S%SNQP"      
 [213] "SPGV%E"       "S%PGTDT"      "SVGQG%TE"     "S%QGTTE"     
 [217] "SYSG%DT"      "S%GYTGE"      "R%GSYE"       "SRG%MNTE"    
 [221] "S%GQGSSYE"    "SGT%NTE"      "S%RQGNSP"     "SD%NYG"      
 [225] "S%GTGRYE"     "SPGG%E"       "SR%NTE"       "S%YRGNTE"    
 [229] "SL%GANTGE"    "S%NTE"        "%GIWVYE"      "S%GIWVYE"    
 [233] "R%GPYE"       "SYLG%E"       "SP%NTE"       "SG%GQET"     
 [237] "%RGNTGE"      "SIS%GE"       "SPGL%YE"    

In [14]:
cols_gliph <- colnames(pbmc_hla_v_j_gliph)[274:ncol(pbmc_hla_v_j_gliph)]

In [15]:
pbmc_hla_v_j_gliph$cyclenum <- as.numeric(gsub("X0", "", pbmc_hla_v_j_gliph$cycle))

In [16]:
pbmc_flowonly <- pca.fx(pbmc_hla_v_j_gliph, c(cols_flow), 2)
pbmc_tcronly <- pca.fx(pbmc_hla_v_j_gliph, c(cols_others, cols_tbv, cols_tbj), 5)
pbmc_tcrhla <- pca.fx(pbmc_hla_v_j_gliph, c(cols_others, cols_tbv, cols_tbj, cols_hla), 5)
pbmc_all <- pca.fx(pbmc_hla_v_j_gliph, c(cols_flow, cols_others, cols_tbv, cols_tbj, cols_hla), 5)

In [17]:
pbmc_gliph <- pca.fx(pbmc_hla_v_j_gliph, c(cols_gliph, cols_tbv, cols_tbj, cols_hla, cols_others, cols_flow), 5)

In [24]:
cfdna_hla_v_j_gliph <- readr::read_rds(paste0(datapath, "/int_cfdna_hla_v_j_gliph.rds"))

In [25]:
rownames(cfdna_hla_v_j_gliph) <- cfdna_hla_v_j_gliph$sample_id

In [26]:
dim(cfdna_hla_v_j_gliph)

[1]  274 1495

In [27]:
table(cfdna_hla_v_j_gliph$Relapse)


  0   1 
226  48 

In [28]:
cols_tbv <- colnames(cfdna_hla_v_j_gliph)[which(grepl("TRBV", colnames(cfdna_hla_v_j_gliph)) ) ]
cols_tbj <- colnames(cfdna_hla_v_j_gliph)[which(grepl("TRBJ", colnames(cfdna_hla_v_j_gliph)) ) ]
cols_flow <- c('ATC', 'Naïve%','SCM%','CM%','EM%','TE%','PD1%','LAG3%','TIM3%')
cols_others <- c("log10shann")
cols_hla <- colnames(cfdna_hla_v_j_gliph)[which(grepl(":0", colnames(cfdna_hla_v_j_gliph)) ) ]

In [29]:
which(grepl("TRBJ", colnames(cfdna_hla_v_j_gliph) ) )

[1] 258 259 260 261 262 263 264 265 266 267 268 269 270

In [31]:
cols_gliph <- colnames(cfdna_hla_v_j_gliph)[271:ncol(cfdna_hla_v_j_gliph)]

In [32]:
cfdna_hla_v_j_gliph$cyclenum <- as.numeric(gsub("X0", "", cfdna_hla_v_j_gliph$cycle))

In [33]:
cfdna_flowonly <- pca.fx(cfdna_hla_v_j_gliph, c(cols_flow), 2)
cfdna_tcronly <- pca.fx(cfdna_hla_v_j_gliph, c(cols_others, cols_tbv, cols_tbj), 2)
cfdna_tcrhla <- pca.fx(cfdna_hla_v_j_gliph, c(cols_others, cols_tbv, cols_tbj, cols_hla), 2)
cfdna_all <- pca.fx(cfdna_hla_v_j_gliph, c(cols_flow, cols_others, cols_tbv, cols_tbj, cols_hla), 2)

In [39]:
cfdna_gliph <- pca.fx(cfdna_hla_v_j_gliph, c(cols_gliph, cols_tbv, cols_tbj, cols_hla, cols_others, cols_flow), 5)
cfdna_gliphonly <- pca.fx(cfdna_hla_v_j_gliph, c(cols_gliph), 5)
cfdna_gliphhla <- pca.fx(cfdna_hla_v_j_gliph, c(cols_gliph, cols_hla), 5)

In [42]:
cfdna_gliphhla[5:6]

[[1]]
Logistic Regression Model

rms::lrm(formula = Relapse ~ sumdist_n, data = mypatients)

                      Model Likelihood     Discrimination    Rank Discrim.    
                            Ratio Test            Indexes          Indexes    
Obs            42    LR chi2      2.21     R2       0.077    C       0.491    
 0             32    d.f.            1     R2(1,42) 0.028    Dxy    -0.019    
 1             10    Pr(> chi2) 0.1370    R2(1,22.9)0.052    gamma  -0.019    
max |deriv| 0.001                          Brier    0.176    tau-a  -0.007    

          Coef    S.E.   Wald Z Pr(>|Z|)
Intercept -0.9566 0.3984 -2.40  0.0164  
sumdist_n -0.2349 0.4321 -0.54  0.5866  


[[2]]
Logistic Regression Model

rms::lrm(formula = Relapse ~ meddist_n, data = mypatients)

                      Model Likelihood     Discrimination    Rank Discrim.    
                            Ratio Test            Indexes          Indexes    
Obs            42    LR chi2      2.57     R2       0.08

# Impute with imputePCA

In [86]:
imputepca.fx <- function(df, cols) {
    mymat <- df[, cols]
    rownames(mymat) <- df[["sample_id"]]
    # remove rows with all NAs
    mymat <- mymat[rowSums(is.na(mymat)) != ncol(mymat), ]
    # rmeove those with only 0s and NAs
    mymat <- mymat[rowSums(mymat, na.rm = TRUE) != 0, ]
    # order df by sample_id
    df <- df[rownames(mymat), ]

    # myvars <- sort(Rfast::colVars(mymat), decreasing = TRUE)
    # message("low variance vars")
    # print(names(myvars)[myvars < 0.000001] )
    # estimatenp <- estim_ncpPCA(mymat, ncp.min = 2, ncp.max = 5, scale = TRUE, method.cv = "Kfold", nbsim = 10, pNA = 0.01, threshold = 1e-4)
    # p0 <- plot(0:5, estimatenp$criterion, xlab = "estimatenp dim", ylab = "MSEP")
    
    res.comp <- imputePCA(mymat, ncp = 5, maxiter = 1000)

    res.pca <- PCA(res.comp$completeObs, quali.sup = NULL, ncp = 5, graph = FALSE) # use default 5 ncp

    # plot PCA with factoextra
    p1 <- fviz_pca_ind(res.pca,
        col.ind = as.vector(df$cancergroup),
        # remove labels
        label = "none",
        repel = TRUE, addEllipses = FALSE
    )
    # scree plot
    p2 <- fviz_eig(res.pca)
    myx <- res.pca$ind$coord
    mydist <- as.matrix(dist(myx)) # use all 5 ncp

    sample_df <- df %>%
        group_by(Patient) %>%
        # order cyclenum
        arrange(cyclenum) %>%
        ungroup()
    pts <- names(which(table(sample_df$Patient) > 1))
    # print(head(sample_df_narm[,1:10]) )
    sample_df$dist <- NA
    for (i in pts) {
        tmp <- sample_df[sample_df$Patient == i, ]
        distances_df <- mydist[tmp$sample_id, tmp$sample_id]
        if (nrow(tmp) == 2) {
            mydiff <- diff(tmp$cyclenum)
            if (mydiff == 1) {
                tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
            }
        }
        if (nrow(tmp) == 3) {
            mydiff <- diff(tmp$cyclenum)
            if (mydiff[1] == 1) {
                tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
            }
            if (mydiff[2] == 1) {
                tmp$dist[3] <- distances_df[tmp$sample_id[2], tmp$sample_id[3]]
            }
        }
        if (nrow(tmp) == 4) {
            mydiff <- diff(tmp$cyclenum)
            if (mydiff[1] == 1) {
                tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
            }
            if (mydiff[2] == 1) {
                tmp$dist[3] <- distances_df[tmp$sample_id[2], tmp$sample_id[3]]
            }
            if (mydiff[3] == 1) {
                tmp$dist[4] <- distances_df[tmp$sample_id[3], tmp$sample_id[4]]
            }
        }
        if (nrow(tmp) == 5) {
            mydiff <- diff(tmp$cyclenum)
            if (mydiff[1] == 1) {
                tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
            }
            if (mydiff[2] == 1) {
                tmp$dist[3] <- distances_df[tmp$sample_id[2], tmp$sample_id[3]]
            }
            if (mydiff[3] == 1) {
                tmp$dist[4] <- distances_df[tmp$sample_id[3], tmp$sample_id[4]]
            }
            if (mydiff[4] == 1) {
                tmp$dist[5] <- distances_df[tmp$sample_id[4], tmp$sample_id[5]]
            }
        }
        sample_df$dist[sample_df$Patient == i] <- tmp$dist
    }
    # message("sample_df")
    # print(head(sample_df))
    allpts <- as.data.frame(table(sample_df$Patient), stringsAsFactors = F)
    sample_df$dist[sample_df$dist == 0] <- NA
    allpts$sumdist <- NA
    allpts$meddist <- NA
    for (i in 1:nrow(allpts)) {
        tmp <- as.data.frame(sample_df[sample_df$Patient == allpts$Var1[i], ])
        allpts$sumdist[i] <- sum(tmp[tmp$dist != 0, "dist"], na.rm = T)
        allpts$meddist[i] <- median(tmp[tmp$dist != 0, "dist"], na.rm = T)
    }
    allpts$sumdist_n <- allpts$sumdist / allpts$Freq
    allpts$meddist_n <- allpts$meddist / allpts$Freq
    # message("allpts")
    # print(head(allpts))
    allpts_no0 <- allpts[allpts$sumdist_n > 0, ]
    allpts_no0 <- allpts_no0[allpts_no0$Freq > 1, ]
    # message("allpts_no0")
    # print(dim(allpts_no0))

    mysample_df <- sample_df[sample_df$Patient %in% allpts_no0$Var1, ]
    # message("mysample_df")
    # print(head(mysample_df))
    # dim(mysample_df)
    mypatients <- mysample_df[!duplicated(mysample_df$Patient), ]
    # message("mypatients")
    # print(head(mypatients))

    mypatients$sumdist_n <- allpts_no0$sumdist_n[match(mypatients$Patient, allpts_no0$Var1)]
    mypatients$sumdist <- allpts_no0$sumdist[match(mypatients$Patient, allpts_no0$Var1)]
    mypatients$meddist_n <- allpts_no0$meddist_n[match(mypatients$Patient, allpts_no0$Var1)]
    mypatients$meddist <- allpts_no0$meddist[match(mypatients$Patient, allpts_no0$Var1)]
    # print(head(mypatients) )
    myfit_sum <- rms::lrm(Relapse ~ sumdist_n , data = mypatients)
    myfit_med <- rms::lrm(Relapse ~ meddist_n , data = mypatients)
    return(list(res.pca, p1, p2, mypatients, myfit_sum, myfit_med))
}


In [101]:
pbmc_hla_v_j_gliph <- readr::read_rds(paste0(datapath, "/int_pbmc_hla_v_j_gliph.rds"))

In [102]:
rownames(pbmc_hla_v_j_gliph) <- pbmc_hla_v_j_gliph$sample_id

In [103]:
dim(pbmc_hla_v_j_gliph)

[1]  534 8829

In [104]:
table(pbmc_hla_v_j_gliph$Relapse)


  0   1 
431 103 

In [105]:
cols_tbv <- colnames(pbmc_hla_v_j_gliph)[which(grepl("TRBV", colnames(pbmc_hla_v_j_gliph)) ) ]
cols_tbj <- colnames(pbmc_hla_v_j_gliph)[which(grepl("TRBJ", colnames(pbmc_hla_v_j_gliph)) ) ]
cols_flow <- c('Naïve%','SCM%','CM%','EM%','TE%','PD1%','LAG3%','TIM3%')
cols_others <- c("log10shann")
cols_hla <- colnames(pbmc_hla_v_j_gliph)[which(grepl(":0", colnames(pbmc_hla_v_j_gliph)) ) ]

In [106]:
cols_gliph <- colnames(pbmc_hla_v_j_gliph)[274:ncol(pbmc_hla_v_j_gliph)]

In [107]:
pbmc_hla_v_j_gliph$cyclenum <- as.numeric(gsub("X0", "", pbmc_hla_v_j_gliph$cycle))

Main

In [ ]:
myvars <- sort(Rfast::colVars(pbmc_hla_v_j_gliph[,c(cols_gliph, cols_hla, cols_flow, cols_others, cols_tbj, cols_tbv)], na.rm = T), decreasing = TRUE)

In [ ]:
rmcols <- names(myvars)[myvars < 0.25] # 1th percentile

In [ ]:
hist(log10(myvars), breaks = 100)

In [ ]:
summary(myvars)

In [ ]:
# cols_gliph <- cols_gliph[which(!cols_gliph %in% rmcols)] 

In [108]:
pbmc_flowonly <- imputepca.fx(pbmc_hla_v_j_gliph, c(cols_flow))

In [115]:
set.seed(777)
pbmc_all <- imputepca.fx(pbmc_hla_v_j_gliph, c(cols_flow, cols_tbv, cols_tbj))

In [116]:
pbmc_all[5:6]

[[1]]
Logistic Regression Model

rms::lrm(formula = Relapse ~ sumdist_n, data = mypatients)

                      Model Likelihood     Discrimination    Rank Discrim.    
                            Ratio Test            Indexes          Indexes    
Obs            71    LR chi2      0.71     R2       0.017    C       0.604    
 0             59    d.f.            1     R2(1,71) 0.000    Dxy     0.208    
 1             12    Pr(> chi2) 0.3992    R2(1,29.9)0.000    gamma   0.209    
max |deriv| 3e-11                          Brier    0.139    tau-a   0.059    

          Coef    S.E.   Wald Z Pr(>|Z|)
Intercept -1.7955 0.4067 -4.42  <0.0001 
sumdist_n  0.0452 0.0515  0.88  0.3804  


[[2]]
Logistic Regression Model

rms::lrm(formula = Relapse ~ meddist_n, data = mypatients)

                      Model Likelihood     Discrimination    Rank Discrim.    
                            Ratio Test            Indexes          Indexes    
Obs            71    LR chi2      0.34     R2       0.00

In [66]:
cfdna_hla_v_j_gliph <- readr::read_rds(paste0(datapath, "/int_cfdna_hla_v_j_gliph.rds"))

In [67]:
rownames(cfdna_hla_v_j_gliph) <- cfdna_hla_v_j_gliph$sample_id

In [68]:
dim(cfdna_hla_v_j_gliph)


[1]  274 1495

In [69]:
table(cfdna_hla_v_j_gliph$Relapse)


  0   1 
226  48 

In [70]:
cols_tbv <- colnames(cfdna_hla_v_j_gliph)[which(grepl("TRBV", colnames(cfdna_hla_v_j_gliph)) ) ]
cols_tbj <- colnames(cfdna_hla_v_j_gliph)[which(grepl("TRBJ", colnames(cfdna_hla_v_j_gliph)) ) ]
cols_flow <- c('ATC', 'Naïve%','SCM%','CM%','EM%','TE%','PD1%','LAG3%','TIM3%')
cols_others <- c("log10shann")
cols_hla <- colnames(cfdna_hla_v_j_gliph)[which(grepl(":0", colnames(cfdna_hla_v_j_gliph)) ) ]

In [71]:
which(grepl("TRBJ", colnames(cfdna_hla_v_j_gliph) ) )

[1] 258 259 260 261 262 263 264 265 266 267 268 269 270

In [72]:
colnames(cfdna_hla_v_j_gliph)[271:ncol(cfdna_hla_v_j_gliph)]

[1] "SLG%E"       "GIWV"        "IWV"         "WVYE"        "IWVY"       
   [6] "S%TGNTE"     "SV%GNTE"     "S%GNYG"      "S%RTGE"      "TGIW"       
  [11] "R%STDT"      "S%GQMNTE"    "SP%MNTE"     "%GTGNTE"     "SPG%TGE"    
  [16] "SA%NTE"      "SLG%NTE"     "S%GSYE"      "RG%GGTE"     "SL%DE"      
  [21] "S%TENTE"     "S%GRTDT"     "S%RGYG"      "SDR%TGE"     "SP%RGNTE"   
  [26] "SPRQG%NTE"   "TG%SYE"      "SP%RNTE"     "S%GVNE"      "SPGP%E"     
  [31] "SY%GTE"      "SP%GYG"      "SPG%GTE"     "SL%GE"       "S%SSYNE"    
  [36] "S%QGATE"     "SAGQ%NTE"    "S%SGE"       "S%PNTE"      "S%SYE"      
  [41] "S%AGANTGE"   "SQDR%T"      "S%MNTE"      "S%GGDG"      "S%SNQP"     
  [46] "S%GYTGE"     "S%GQGSSYE"   "SL%GANTGE"   "S%NTE"       "SP%NTE"     
  [51] "SRGT%YE"     "SL%GATNEK"   "SLT%NTE"     "R%GMNTE"     "S%GTE"      
  [56] "%AGNTE"      "S%GGTE"      "S%GGNE"      "R%TGGTE"     "S%RGPGE"    
  [61] "S%QET"       "S%QNSNQP"    "SPG%YE"      "S%GGTGE"     "SPG%GE"     
  [66] "RQ%GNQP"     "SFG%MNTE"    "S%GTGE"      "SR%GMNTE"    "S%QGGTE"    
  [71] "%GLNYG"      "GS%NTE"      "SSS%E"       "SLGL%YE"     "S%TGGTE"    
  [76] "SFGQ%NTE"    "S%GTGGTGE"   "SLT%MNTE"    "SL%GYE"      "SYS%GE"     
  [81] "SL%GNTE"     "S%GSNQP"     "SIG%GSYE"    "S%GDE"       "SLG%TE"     
  [86] "S%SNTE"      "SS%YE"       "S%NTGE"      "SL%SGE"      "SLGG%E"     
  [91] "SPG%LNTE"    "SY%GTTE"     "SLG%NYG"     "SL%GNE"      "S%GGNQP"    
  [96] "S%TGMNTE"    "SL%TE"       "SL%NE"       "SY%GYE"      "SL%SGGE"    
 [101] "S%RGNE"      "SL%GAGGNT"   "S%RGMNTE"    "S%GQVNTE"    "SL%YE"      
 [106] "%TGGNTE"     "SLD%NTE"     "SLTSG%YNE"   "SL%GTE"      "S%RGTE"     
 [111] "SLGQ%NTE"    "SL%ENTE"     "SPS%TE"      "S%GDNE"      "SLR%NE"     
 [116] "SLGQGA%NTE"  "STGTG%E"     "SLGQ%NEK"    "S%QGNE"      "%VLGDE"     
 [121] "SLTG%TE"     "GQGN%E"      "S%SPGGYE"    "SL%GFNQP"    "SPGQG%DG"   
 [126] "S%SGYE"      "SLG%NE"      "SQDR%G"      "SSG%SGNT"    "SLV%GMNTE"  
 [131] "SPN%E"       "SLG%SYE"     "%LGLNTE"     "SPG%YG"      "S%TGYE"     
 [136] "S%GSTDT"     "SV%GGQET"    "S%GGYG"      "S%GGNT"      "S%SGNT"     
 [141] "S%SMNTE"     "SYS%E"       "S%QGAYE"     "SV%GADE"     "S%GGTTE"    
 [146] "SL%GGNTE"    "SPGTG%YG"    "SL%TDT"      "SL%TGIWVYE"  "SP%EVNTE"   
 [151] "S%RTGSYE"    "%LGNE"       "S%YNE"       "SF%SSYE"     "S%GSE"      
 [156] "SLG%GQP"     "SLVSG%E"     "SL%QGAYE"    "S%QGMNTE"    "SRQG%GTE"   
 [161] "SL%SNQP"     "SL%GTGE"     "SVTG%TE"     "SQD%NQP"     "SLGTGL%G"   
 [166] "%GNTE"       "SSG%TYE"     "SP%TGNYG"    "SL%GGNTGE"   "SYR%E"      
 [171] "SSR%SYE"     "SSG%E"       "S%GAGE"      "SLVG%E"      "S%WNTE"     
 [176] "SGG%YG"      "S%GQGVYE"    "SLR%GE"      "S%GQET"      "SL%RSE"     
 [181] "SPR%NTE"     "SPRT%STDT"   "SLG%ET"      "SL%RNQP"     "S%GLNTE"    
 [186] "SL%GSYE"     "S%RGSYE"     "SLGQG%NTE"   "SL%TGE"      "S%GAGNT"    
 [191] "SSGLAT%E"    "SLG%TNYG"    "%ISGANV"     "SV%GNQP"     "SS%RGKET"   
 [196] "SPSP%NTE"    "SLEGL%TGE"   "S%TGE"       "SL%AGNT"     "SP%TE"      
 [201] "SLG%TDT"     "S%TYE"       "SF%DT"       "S%GGSYE"     "SY%GE"      
 [206] "STGD%"       "RGTG%TE"     "S%GPYE"      "SP%TGE"      "SLE%TE"     
 [211] "SL%SE"       "STR%TE"      "S%SGKNTE"    "GQGNY%"      "SQ%SGNT"    
 [216] "SLGG%QP"     "RG%LNTE"     "RTG%NTE"     "S%GDSSYE"    "S%DSNSP"    
 [221] "SFTG%TE"     "SL%QGGTE"    "S%RLNTE"     "S%DSGGYNE"   "SYSG%"      
 [226] "SQDRR%"      "SPGT%NTE"    "SQ%MNTE"     "SLGQ%NYG"    "SPG%NQP"    
 [231] "S%STDT"      "SLGV%E"      "S%GQGTE"     "SLA%NTGE"    "SV%MNTE"    
 [236] "SLAGG%VYE"   "S%GGGNSP"    "S%GQRNTE"    "SL%GMNTE"    "SSP%AGNTE"  
 [241] "SLRPP%I"     "SVG%QAGDT"   "SFKRGFSG%NV" "SLV%GE"      "S%TGGNTE"   
 [246] "S%NRNTE"     "SFG%NQP"     "SPG%VNTE"    "SAGTGG%YE"   "S%TSGETDT"  
 [251] "SDS%TGE"     "G%MNTE"      "SL%GTYNE"    "SLR%TE"      "SRGG%YE"    
 [256] "S%SGENTGE"   "SVEG%TE"     "SLSG%YNE"    "SS%ISGNT"    "S%GGSSYE"   

In [73]:
cols_gliph <- colnames(cfdna_hla_v_j_gliph)[271:ncol(cfdna_hla_v_j_gliph)]

In [74]:
cfdna_hla_v_j_gliph$cyclenum <- as.numeric(gsub("X0", "", cfdna_hla_v_j_gliph$cycle))

In [ ]:
# cfdna_flowonly <- imputepca.fx(cfdna_hla_v_j_gliph, c(cols_flow))

In [ ]:
# cfdna_tcrhla <- imputepca.fx(cfdna_hla_v_j_gliph, c(cols_others, cols_tbv, cols_tbj))

In [78]:
cfdna_all <- imputepca.fx(cfdna_hla_v_j_gliph, c(cols_flow, cols_others, cols_tbv))

In [80]:
cfdna_all[5:6]

[[1]]
Logistic Regression Model

rms::lrm(formula = Relapse ~ sumdist_n, data = mypatients)

                      Model Likelihood     Discrimination    Rank Discrim.    
                            Ratio Test            Indexes          Indexes    
Obs            71    LR chi2      0.61     R2       0.014    C       0.600    
 0             57    d.f.            1     R2(1,71) 0.000    Dxy     0.199    
 1             14    Pr(> chi2) 0.4346    R2(1,33.7)0.000    gamma   0.199    
max |deriv| 5e-05                          Brier    0.157    tau-a   0.064    

          Coef    S.E.   Wald Z Pr(>|Z|)
Intercept -1.1315 0.4580 -2.47  0.0135  
sumdist_n -0.1340 0.1829 -0.73  0.4638  


[[2]]
Logistic Regression Model

rms::lrm(formula = Relapse ~ meddist_n, data = mypatients)

                      Model Likelihood     Discrimination    Rank Discrim.    
                            Ratio Test            Indexes          Indexes    
Obs            71    LR chi2      0.24     R2       0.00

# Impute with rms

In [ ]:
pacman::p_load(rms)

In [ ]:
dim(sample_df)

In [ ]:
mycols <- gsub("%", "_p", mycols)

In [ ]:
colnames(sample_df) <- gsub("%", "_p", colnames(sample_df))

In [ ]:
colnames(sample_df)

In [ ]:
ipc_r <- prcomp( formula( paste("~", paste(mycols[!grepl("TIM3", mycols)]) ) ),
data = sample_df)

In [ ]:
ptrans <- transcan(  as.formula( paste("~", paste(mycols[!grepl("TIM3", mycols)], collapse = "+") ) ) ,
imputed=TRUE , transformed =TRUE , data= sample_df , pl= FALSE , pr= FALSE, iter.max = 100)

In [ ]:
trans <- ptrans$transformed

In [ ]:
imp <- impute(ptrans , data= sample_df , list.out= TRUE)

In [ ]:
names(imp)

In [ ]:
NAvars <- all.vars( as.formula( paste("~", paste(names(imp), collapse = "+") ) ))

In [ ]:
NAvars

In [ ]:
for(x in NAvars ) sample_df[[x]] <- imp[[x]]

In [ ]:
table(sample_df$Relapse)

In [ ]:
trans_fit <- lrm(Relapse ~ trans, data = sample_df)

In [ ]:
myf <- as.formula( paste("Relapse ~", paste(mycols[!grepl("TIM3", mycols)], collapse = "+") ) )

In [ ]:
raw_fit <- lrm( myf, data = sample_df ) 

In [ ]:
AIC(trans_fit)
AIC(raw_fit)

In [ ]:
pca_raw <- prcomp( sample_df[,mycols[!grepl("TIM3", mycols)] ])$x[,1:5]

In [ ]:
pca_trans <- prcomp( trans[,mycols[!grepl("TIM3", mycols)] ])$x[,1:5]

In [ ]:
pcaraw_fit <- lrm( Relapse ~ pca_raw, data = sample_df ) 
pcatrans_fit <- lrm(Relapse ~ pca_trans, data = sample_df )

In [ ]:
AIC(pcaraw_fit)
AIC(pcatrans_fit)

# Backup

In [ ]:
mymat <- pbmc_hla_v_j_gliph[, c(66,200:270)]
mymat <- cbind(mymat, pbmc_hla_v_j_gliph[,c('ATC', 'Naïve%','SCM%','CM%','EM%','TE%','PD1%','LAG3%','TIM3%')])
# mymat <- pbmc_hla_v_j_gliph[,c('ATC', 'Naïve%','SCM%','CM%','EM%','TE%','PD1%','LAG3%','TIM3%')]
rownames(mymat) <- pbmc_hla_v_j_gliph$sample_id

In [ ]:
head(mymat, 10)

In [ ]:
# remove rows with all NAs
mymat <- mymat[ rowSums(is.na(mymat)) != ncol(mymat), ]
# rmeove those with only 0s and NAs
mymat <- mymat[ rowSums(mymat, na.rm = TRUE) != 0, ]

In [ ]:
# sample_df <- as.data.frame(sample_df)
rownames(pbmc_hla_v_j_gliph) <- pbmc_hla_v_j_gliph$sample_id

In [ ]:
pbmc_hla_v_j_gliph <- pbmc_hla_v_j_gliph[ rownames(mymat), ]

In [ ]:
dim(pbmc_hla_v_j_gliph)

In [ ]:
dim(mymat)

In [ ]:
pbmc_hla_v_j_gliph <- pbmc_hla_v_j_gliph[ rownames(mymat), ]

In [ ]:
pbmc_hla_v_j_gliph$cyclenum <- as.numeric(gsub("X0", "", pbmc_hla_v_j_gliph$cycle))

# PCA

In [ ]:
mymat_narm <- mymat[ rowSums(is.na(mymat)) == 0, ]
pbmc_hla_v_j_gliph_narm <- pbmc_hla_v_j_gliph[ rownames(mymat_narm), ]

In [ ]:
head(pbmc_hla_v_j_gliph_narm)

In [ ]:
myvars <- sort( Rfast::colVars(mymat_narm) , decreasing = TRUE)

In [ ]:
mymat_narm <- mymat_narm[ , names(myvars)[ myvars > 0]]

In [ ]:
dim(mymat_narm)
dim(mymat)


In [ ]:
#PCA
pca <- prcomp(mymat_narm, center = TRUE, scale. = TRUE)

In [ ]:
# plot PCA with factoextra
viz_pca <- fviz_pca_ind(pca,
    col.ind = as.vector(pbmc_hla_v_j_gliph_narm$cancergroup),
    # gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
    # remove labels
    label = "none",
    repel = TRUE, addEllipses = FALSE
)

In [ ]:
viz_pca# + coord_cartesian(xlim = c(-5, 5), ylim = c(-5, 5) )

In [ ]:
dim(mymat_narm)

In [ ]:
sum( summary(pca)$importance[2,1:5] )

In [ ]:
# scree plot
fviz_eig(pca)

In [ ]:
myx <- pca$x

In [ ]:
mydist <- as.matrix(dist(myx[,1:5]))

In [ ]:
head(mydist)

In [ ]:
# pbmc_hla_v_j_gliph_narm$cyclenum <- as.numeric(gsub("X0", "", pbmc_hla_v_j_gliph_narm$cycle))

In [ ]:
sample_df_narm <- pbmc_hla_v_j_gliph_narm %>% group_by(Patient) %>% 
# order cyclenum
arrange(cyclenum) %>%
ungroup()

In [ ]:
pts <- names(which(table(sample_df_narm$Patient) > 1 ))

In [ ]:
sample_df_narm$dist <- NA

In [ ]:
sample_df_narm <- pbmc_hla_v_j_gliph_narm

In [ ]:
for (i in pts) {
    tmp <- sample_df_narm[sample_df_narm$Patient == i, ]
    # tmp <- tmp[ order(tmp$cyclenum, decreasing = F), ]
    distances_df <- mydist[tmp$sample_id, tmp$sample_id]
    if (nrow(tmp) == 2) {
        mydiff <- diff(tmp$cyclenum)
        if (mydiff == 1) {
            tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
        }
    }
    if (nrow(tmp) == 3) {
        mydiff <- diff(tmp$cyclenum)
        if (mydiff[1] == 1) {
            tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
        }
        if (mydiff[2] == 1) {
            tmp$dist[3] <- distances_df[tmp$sample_id[2], tmp$sample_id[3]]
        }
    }
    if (nrow(tmp) == 4) {
        mydiff <- diff(tmp$cyclenum)
        if (mydiff[1] == 1) {
            tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
        }
        if (mydiff[2] == 1) {
            tmp$dist[3] <- distances_df[tmp$sample_id[2], tmp$sample_id[3]]
        }
        if (mydiff[3] == 1) {
            tmp$dist[4] <- distances_df[tmp$sample_id[3], tmp$sample_id[4]]
        }
    }
    if (nrow(tmp) == 5) {
        mydiff <- diff(tmp$cyclenum)
        if (mydiff[1] == 1) {
            tmp$dist[2] <- distances_df[tmp$sample_id[1], tmp$sample_id[2]]
        }
        if (mydiff[2] == 1) {
            tmp$dist[3] <- distances_df[tmp$sample_id[2], tmp$sample_id[3]]
        }
        if (mydiff[3] == 1) {
            tmp$dist[4] <- distances_df[tmp$sample_id[3], tmp$sample_id[4]]
        }
        if (mydiff[4] == 1) {
            tmp$dist[5] <- distances_df[tmp$sample_id[4], tmp$sample_id[5]]
        }
    }
    sample_df_narm$dist[sample_df_narm$Patient == i] <- tmp$dist
}


In [ ]:
# sample_df_narm$dist[is.na(sample_df_narm$dist)] <- 0

In [ ]:
head(sample_df_narm)

In [ ]:
allpts <- as.data.frame(table(sample_df_narm$Patient), stringsAsFactors = F)

In [ ]:
# sample_df_narm$Relapse[is.na(sample_df_narm$Relapse)] <- 0

In [ ]:
# t.test(sample_df_narm$dist[sample_df_narm$dist != 0] ~ sample_df_narm$Relapse[sample_df_narm$dist != 0])

In [ ]:
sample_df_narm[ sample_df_narm$Patient == "CHP_351",]

In [ ]:
sample_df_narm$dist[ sample_df_narm$dist == 0] <- NA

In [ ]:
allpts$sumdist <- NA
for(i in 1:nrow(allpts)){
    tmp <- as.data.frame(sample_df_narm[ sample_df_narm$Patient == allpts$Var1[i],] )
    allpts$sumdist[i] <- sum(tmp[ tmp$dist != 0, "dist"], na.rm = T)
}

In [ ]:
allpts$sumdist_n <- allpts$sumdist / allpts$Freq

In [ ]:
head(allpts, 10)

In [ ]:
allpts_no0 <- allpts[ allpts$sumdist_n > 0,]

In [ ]:
dim(allpts_no0)

In [ ]:
mysample_df <- sample_df_narm[ sample_df_narm$Patient %in% allpts_no0$Var1,]

In [ ]:
dim(mysample_df)

In [ ]:
mypatients <- mysample_df[!duplicated(mysample_df$Patient),]

In [ ]:
dim(mypatients)

In [ ]:
head(mypatients)

In [ ]:
head(allpts_no0)

In [ ]:
mypatients$sumdist_n <- allpts_no0$sumdist_n[ match(mypatients$Patient, allpts_no0$Var1)]
mypatients$sumdist <- allpts_no0$sumdist[ match(mypatients$Patient, allpts_no0$Var1)]

In [ ]:
# tmp$Relapse[ is.na(tmp$Relapse) ] <- 0

In [ ]:
dim(mypatients)

In [ ]:
head(mypatients, 10)

In [ ]:
myfit <- rms::lrm(Relapse ~ sumdist_n, data = mypatients)

In [ ]:
print(myfit)

In [ ]:
anova(myfit)

In [ ]:
mypatients <- mypatients[order(mypatients$sumdist_n),]

In [ ]:
makematrix_4hm <- function(df, var, name) {
    mymat <- df[[var]]
    names(mymat) <- df[["Patient"]]
    mymat <- t(as.matrix(mymat))
    rownames(mymat) <- name
    return(mymat)
}

In [ ]:
mycohort <- makematrix_4hm(mypatients, "Disease_type", "disease")
mysample <- makematrix_4hm(mypatients, "cancergroup", "cancergroup")
mygroup <- makematrix_4hm(mypatients, "Relapse", "Relapse")
# myatc <- makematrix_4hm(tmp, "ATC", "ATC")
myage <- makematrix_4hm(mypatients, "Age", "Age")

In [ ]:
age_col <- c("blue", "red")
# atc_col <- c("blue", "red")

In [ ]:
group_col <- c("blue", "red", "green")
names(group_col) <- names(table(mypatients$Relapse))

In [ ]:
# cohort_col <- c("blue", "red", "green", "orange", "purple", "yellow", "black", "grey", "pink", 
# "brown", "cyan", "magenta", "navy", "maroon", "gray","gold", "darkgreen") 
# names(cohort_col) <- names(table(allpbmcs$Disease_type))

In [ ]:
type_col <- c("blue", "red", "green", "orange", "purple")
names(type_col) <- names(table(mypatients$cancergroup))

In [ ]:
age_hm <- Heatmap(myage, col = age_col,  height = unit(1,"cm"))
# atc_hm <- Heatmap(myatc, col = atc_col,  height = unit(1,"cm"))

In [ ]:
cohort_hm <- Heatmap(mycohort, col = cohort_col, height = unit(1,"cm"))
sample_hm <- Heatmap(mysample, col = type_col, height = unit(1,"cm"))
group_hm <- Heatmap(mygroup, col = group_col,  height = unit(1,"cm"))

In [ ]:
# length(unique(sample_df_narm$Patient))

In [ ]:
# tail(sample_df_narm[, c("sample_id", "cycle", "dist")])

In [ ]:
# pivot_wider
 
wider_df <- tidyr::pivot_wider(mysample_df, names_from = cycle, values_from = dist, id_cols = Patient) 

wider_df[ wider_df$Patient == "CHP_350",]

In [ ]:
wider_df <- as.data.frame(wider_df)
rownames(wider_df) <- wider_df$Patient
wider_df$Patient <- NULL
wider_df$X01 <- NULL

In [ ]:
head(wider_df)

In [ ]:
hm1 <- Heatmap(t(scale(wider_df)), cluster_columns = F, name = "dists")

In [ ]:
dim(wider_df)

In [ ]:
hm <- Heatmap(t(scale(mypatients$sumdist_n)), cluster_columns = F)

In [ ]:
dim(mypatients)

In [ ]:
hm %v% hm1 %v% sample_hm %v% group_hm %v% age_hm 

In [ ]:
ls()

In [ ]:
rm(tmp, wider_df, pts, myx, mysample, mysample_df, allpts, allpts_no0, distances_df, hm, hm1, i, mydiff, mydist, myfit, mymat_narm, pca, sample_df_narm)